In [360]:
using Plots
using LaTeXStrings
using SparseArrays

using BandedMatrices
using LinearSolve

In [361]:
# Define functions for the current sheet F, F'' 
F(x) = tanh(x)
ddF(x) = -2 * tanh(x) * sech(x)^2

ddF (generic function with 1 method)

In [362]:
function create_grid(domain::Tuple, n::Int, λ::Rational) 
    # λ ∈ [-1, 1]
    a, b = domain
    # compute spacing on domain: (0, 1)
    u = [((j) / (n-1)) + (λ / (2*π)) * sin(2*π*(j) / (n-1)) for j in 0:n-1]
    # now apply a linear tranformation to go from (0, 1) -> (a, b)
    return [a + (b - a) * k for k in u]
end

create_grid (generic function with 1 method)

In [363]:
# Function to compute finite difference weights
function stencil(x::AbstractVector{<:Real}, x₀::Real, m::Integer, p::Integer=2)
    ℓ = 0:p
    A = @. (x' - x₀)^ℓ / factorial(ℓ)
    b = ℓ .== m
    return A \ b
end

stencil (generic function with 2 methods)

In [364]:
function createA(x, S, gam, bbgam, khat, k, v, F, ddF, p::Integer=2)
    N = length(x)
    A = spzeros(6*N, 6*N)
    # A = zeros(ComplexF64, 6*N, 6*N)  # Matrix to accommodate ψ and φ for m-1, m, m+1

    # Function to insert weights into the matrix
    function insert_weights!(A, idx, weights, offset, factor=1.0)
        for (j, w) in enumerate(weights)
            if idx + (j-2) * 6 + offset > 0 && idx + (j-2) * 6 + offset <= size(A, 1)
                A[idx, idx + (j-2) * 6 + offset] += factor * w
            end
        end
    end

    for i in 1:N
        # Define the indices for the solution vector
        idx_m1_psi, idx_m1_phi = 6*(i-1) + 1, 6*(i-1) + 2
        idx_m_psi, idx_m_phi = 6*(i-1) + 3, 6*(i-1) + 4
        idx_mp1_psi, idx_mp1_phi = 6*(i-1) + 5, 6*(i-1) + 6

        # Compute stencil weights for the first, second, and third derivatives at x[i]
        if i == 1
            w_1st = stencil(x[i:i+2], x[i], 1, p)
            w_2nd = stencil(x[i:i+2], x[i], 2, p)
            w_3rd = stencil(x[i:i+2], x[i], 3, p)
        elseif i == N
            w_1st = stencil(x[i-2:i], x[i], 1, p)
            w_2nd = stencil(x[i-2:i], x[i], 2, p)
            w_3rd = stencil(x[i-2:i], x[i], 3, p)
        else
            w_1st = stencil(x[i-1:i+1], x[i], 1, p)
            w_2nd = stencil(x[i-1:i+1], x[i], 2, p)
            w_3rd = stencil(x[i-1:i+1], x[i], 3, p)
        end

        # ODEs for ψ_{m-1} and φ_{m-1}
        insert_weights!(A, idx_m1_psi, w_2nd, -6, 1)  # ψ_{m-1}
        A[idx_m1_psi, idx_m1_psi] -= S * gam[1]  # ψ_{m-1, i}
        A[idx_m1_psi, idx_m1_phi] = S * gam[1] * F(x[i])  # Coupling term with φ_{m-1, i}
        insert_weights!(A, idx_m1_psi, w_1st, -6, -S * v * (1 - khat / k[2]))  # First derivative term for ψ_{m}

        insert_weights!(A, idx_m1_phi, w_2nd, -6, 1)  # φ_{m-1}
        A[idx_m1_phi, idx_m1_phi] -= k[1]^2  # φ_{m-1, i}
        insert_weights!(A, idx_m1_phi, w_2nd, -6, k[1]^2 / gam[1]^2)  # ψ_{m-1}
        A[idx_m1_phi, idx_m1_psi] += -k[1]^4 / gam[1]^2 - k[1]^2 * ddF(x[i]) / gam[1]^2
        insert_weights!(A, idx_m1_phi, w_3rd, -6, -v * k[1] * bbgam[2] / (gam[1] * k[2]))  # Coupling term with φ_{m, i}

        # ODEs for ψ_{m} and φ_{m}
        insert_weights!(A, idx_m_psi, w_2nd, -6, 1)  # ψ_{m}
        A[idx_m_psi, idx_m_psi] -= S * gam[2]  # ψ_{m, i}
        A[idx_m_psi, idx_m_phi] = S * gam[2] * F(x[i])  # Coupling term with φ_{m, i}
        insert_weights!(A, idx_m_psi, w_1st, -6, -S * v * (1 - khat / k[1]))  # First derivative term for ψ_{m-1}
        insert_weights!(A, idx_m_psi, w_1st, 6, -S * v * (1 + khat / k[3]))  # First derivative term for ψ_{m+1}

        insert_weights!(A, idx_m_phi, w_2nd, -6, 1)  # φ_{m}
        A[idx_m_phi, idx_m_phi] -= k[2]^2  # φ_{m, i}
        insert_weights!(A, idx_m_phi, w_2nd, -6, k[2]^2 / gam[2]^2)  # ψ_{m}
        A[idx_m_phi, idx_m_psi] += -k[2]^4 / gam[2]^2 - k[2]^2 * ddF(x[i]) / gam[2]^2
        insert_weights!(A, idx_m_phi, w_3rd, -6, -v * k[2] * bbgam[1] / (gam[2] * k[1]))  # Coupling term with φ_{m-1, i}
        insert_weights!(A, idx_m_phi, w_3rd, 6, -v * k[2] * bbgam[3] / (gam[2] * k[3]))  # Coupling term with φ_{m+1, i}

        # ODEs for ψ_{m+1} and φ_{m+1}
        insert_weights!(A, idx_mp1_psi, w_2nd, -6, 1)  # ψ_{m+1}
        A[idx_mp1_psi, idx_mp1_psi] -= S * gam[3]  # ψ_{m+1, i}
        A[idx_mp1_psi, idx_mp1_phi] = S * gam[3] * F(x[i])  # Coupling term with φ_{m+1, i}
        insert_weights!(A, idx_mp1_psi, w_1st, -6, -S * v * (1 - khat / k[2]))  # First derivative term for ψ_{m}

        insert_weights!(A, idx_mp1_phi, w_2nd, -6, 1)  # φ_{m+1}
        A[idx_mp1_phi, idx_mp1_phi] -= k[3]^2  # φ_{m+1, i}
        insert_weights!(A, idx_mp1_phi, w_2nd, -6, k[3]^2 / gam[3]^2)  # ψ_{m+1}
        A[idx_mp1_phi, idx_mp1_psi] += -k[3]^4 / gam[3]^2 - k[3]^2 * ddF(x[i]) / gam[3]^2
        insert_weights!(A, idx_mp1_phi, w_3rd, -6, -v * k[3] * bbgam[2] / (gam[3] * k[2]))  # Coupling term with φ_{m, i}

        # Construct boundary matrices (BandedMatrices)
        # B_left = BandedMatrix(0 => fill(1.0, 6))  # Left boundary matrix
        # B_right = BandedMatrix(0 => fill(1.0, 6))   # Right boundary matrix

        # Apply boundary matrices to A
        # A[1:6, 1:6] = Matrix(B_left)
        # A[end-5:end, end-5:end] = Matrix(B_right)
    end

    return A
end

createA (generic function with 2 methods)

In [365]:
function createUV(A)
    U = A[7:end-5, 7:end-5]; # U starts at ψ_{m-1, 2}

    # now we set ψ_{m-1, 2} = 1 which creates inhomogenenous terms
    V = convert(Vector, -U[:, 1]) # V is the negative of the ψ_{m-1, 2} column
    U_rem = U[:, 2:end] # now we remove the ψ_{m-1, 2} column 

    # note: size(U_rem) should be 6*n - 12 as expected
    return U_rem, V
end

createUV (generic function with 1 method)

In [366]:
function LinSolveUV(U, V)
    # solve the resulting linear system
    prob = LinearProblem(U, V)
    sol = solve(prob)
    Y = sol.u
    return Y
end

LinSolveUV (generic function with 1 method)

In [367]:
# Define constants
khat = 1
k = [0.25, 0.5, 0.25] # k bar: {m-1, m, m+1}

γ₋ = 0.013
γ = 0.01316
γ₊ = 0.013

gam = [γ₋, γ, γ₊] # gamma bar: {m-1, m, m+1}
bbgam = gam # gamma bar bar: {m-1, m, m+1}

S = 1000
v = 0.01; # some small number

In [368]:
dom = (-10, 10)
n = 200
mult = 2//3

x = create_grid(dom, n, mult);
A = createA(x, S, gam, bbgam, khat, k, v, F, ddF, 2);
# U, V = createUV(A);
# Y = LinSolveUV(U, V);

In [370]:
#=

ɸm1 = Y[1:6:end]; # length: n-2, start: x₂, end: x_{n-1}
ψ = Y[2:6:end]; # length: n-2, start: x₂, end: x_{n-1}
ɸ = Y[3:6:end]; # length: n-2, start: x₂, end: x_{n-1}
ψp1 = Y[4:6:end]; # length: n-2, start: x₂, end: x_{n-1}
ɸp1 = Y[5:6:end]; # length: n-2, start: x₂, end: x_{n-1}
x_interior = x[2:end-1];

ψm1 = Y[6:6:end]; # length: n-2, start: x₃, end: x_{n}
x_end = x[3:end];

println("Should be near 0: ", real.(ψm1[end]))

theme(:dao::Symbol)
plot(x_interior, [real.(ɸm1), real.(ψ), real.(ɸ), real.(ψp1), real.(ɸp1)], 
    labels=[L"$\phi_{m-1}$" L"$\psi_{m}$" L"$\phi_{m}$" L"$\psi_{m+1}$" L"$\phi_{m+1}$"],
    lw=2, 
    markersize=2.5)
plot!(x_end, real.(ψm1), label=L"$\psi_{m-1}$", 
    lw=3,
    markersize=2.5, 
    legend=:best)

=#